# Задание 1

Реализуйте стохастический градиентный спуск для решения линейной регрессии. Исследуйте сходимость с разным размером батча (1 - SGD, 2, $\ldots$, $n - 1$ - Minibatch GD, $n$ - GD из предыдущей работы).

In [5]:
import sys
sys.path.append('..' + '/')

from helper import *

In [6]:
from sklearn.linear_model import LinearRegression

class get_model_LinearRegression:
    def __init__(self, batch_size=None):
        self.coef_ = []
        self.intercept_ = None
        self.batch_size = batch_size

    def fit(self, X_train, y_train, epsilon=1e-5, learning_rate=0.01, max_epochs=1000, batch_size=1, apply_min=False):
        X = X_train 
        y = y_train

        if self.batch_size is not None:
            batch_size = self.batch_size

        def mse_loss(w):
            y_pred = np.dot(X, w)
            mse = np.mean((y_pred - y)**2)
            return mse

        x0 = np.zeros(X.shape[1], dtype=float)

        point, _, _ = stochastic_gradient_descent(mse_loss, x0, epsilon=epsilon, learning_rate=learning_rate, max_epochs=max_epochs, batch_size=batch_size, apply_min=apply_min)

        self.coef_ = point[:-1]
        self.intercept_ = point[-1] 

    def predict(self, X_test):
        y_pred = []

        for i in range(len(X_test)):
            y = self.intercept_ + sum([self.coef_[j] * float(X_test[i][j]) for j in range(len(self.coef_))])
            y_pred.append(y)
        return y_pred

In [21]:
from sklearn.datasets import fetch_openml
boston = fetch_openml(name='boston', version=1, as_frame=True , parser='liac-arff')


results = []

for i in range(1, boston.data.values.shape[1] + 1):
    model = get_model_LinearRegression(batch_size=i)
    results.append(train_test_print_model(model, boston, print_result=False, view_graphics=False))


In [22]:
print(results)

[(45.32101453393082, 0.3819899563227275), (84.79164837326567, -0.1562426581469749), (89.78954710098428, -0.2243954045666181), (63.036807232677894, 0.1404124468136838), (179.03025399211015, -1.4413066703610893), (52.873964070031505, 0.27899582168783), (170.12961671358372, -1.3199350883295473), (116.39504659179306, -0.587195444933329), (60.85565188927208, 0.17015529178102096), (122.78052186825185, -0.6742695736810413), (1136.5844597812195, -14.498783927407764), (535.0780392156863, -6.296473959974136), (254.72073031516558, -2.473443198181838)]
